In [ ]:
import pandas as pd
from archiv.utils import create_bestand

In [ ]:
file = 'legacy_data/Verhandlungsschriften_neu.xlsx'
res_type_scheme, _ = SkosConceptScheme.objects.get_or_create(dc_title='res_type')
df = pd.read_excel(file).fillna('False')
df['Teil von Bestand'] = df[['Archiv', 'Bestand']].apply(lambda x: '; '.join(x), axis=1)
df['abstract'] = df[['Anfrage', 'Antwort der Kommission']].apply(lambda x: ' ANTWORT DER KOMMISSION: '.join(x), axis=1)

In [ ]:
df

In [ ]:
for i, row in df.iterrows():
    signatur = "|".join(
            [
                str(row.get('Archiv', default="Archiv")),
                str(row.get('Bestand', default="Bestand")),
                str(row.get('Nummer', default="Nummer")), 
                str(row.get('Unterpunkt', default="Unterpunkt")),
                str(row.get('Seite', default="Seite")), 
            ]
        )
    if row['Teil von Bestand'] != 'False':
        item, _ = ArchResource.objects.get_or_create(
            signature=signatur
        )
        try:
            bestand = create_bestand(row['Teil von Bestand'], split=";")[-1]
            item.location = bestand
        except Exception as e:
            print(e)
        if row['Anfrage'] != 'False':
            item.title = row['Anfrage'][:150]
        if row['abstract'] != 'False':
            item.abstract = row['Anfrage'][:150]
        if row['Start Date'] != 'False':
            try:
                item.not_before = pd.to_datetime(row['Start Date'])
            except Exception as e:
                print(e)
        if row['Ort'] != 'False':
            for x in row['Ort'].split(';'):
                try:
                    pl = Place.objects.get(name=x.strip())
                except Exception as e:
                    pl = None
                    print(e, x.strip())
                if pl:
                    item.mentioned_place.add(pl)
        if row['Institution'] != 'False':
            for x in row['Institution'].split(';'):
                try:
                    pl = Institution.objects.get(written_name=x.strip())
                except Exception as e:
                    pl = None
                    print(e, x.strip())
                if pl:
                    item.mentioned_inst.add(pl)
        if row['Person'] != 'False':
            for x in row['Person'].split(';'):
                try:
                    pl = Person.objects.get(written_name=x.strip())
                except Exception as e:
                    pl = None
                    print(e, x.strip())
                if pl:
                    item.mentioned_person.add(pl)
    item.save()
            
        

In [ ]:
ArchResource.objects.filter(signature__startswith='ÖStA/AdR|Staatskanzlei').delete()